In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install flask_restful
!pip install werkzeug
!pip install flask-cors
from flask import Flask ,request,jsonify, send_file
from flask_ngrok import run_with_ngrok
from flask_restful import Resource, Api, reqparse
import subprocess
from flask_cors import CORS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.4.1
!pip install ffmpeg

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
import shutil
shutil.copytree("/content/drive/MyDrive/first", "/content/first")

'/content/first'

In [ ]:
cp "/content/drive/MyDrive/DemoConfig/fashion-tachi-First_order.yaml"  /content/first/config

In [ ]:
cp "/content/drive/MyDrive/DemoConfig/mgif-taichi_FirstOrderModel.yaml"  /content/first/config

In [ ]:
cd /content/first

/content/first


In [ ]:
from first.demo import load_checkpoints
generatorFashion, kp_detectorFashion = load_checkpoints(config_path='/content/first/config/fashion-tachi-First_order.yaml', 
                            checkpoint_path='/content/drive/MyDrive/Fashion_Taichi_First_order/00000182-checkpoint.pth.tar')

In [ ]:
generatorGif, kp_detectorGif = load_checkpoints(config_path='/content/first/config/mgif-taichi_FirstOrderModel.yaml', 
                            checkpoint_path='/content/drive/MyDrive/mgif-train-fomm/00000181-checkpoint.pth.tar')

In [ ]:
server = Flask (__name__)
run_with_ngrok(server)
CORS(server)


@server.route('/post', methods=['POST'])
def get_Image_Video():
  pixel = 256
  print("Ana fel function")
  image = request.files['Image']
  video = request.files['Video']
  model = request.form.get('Model') # Fetch the radio button value
  dataset = request.form.get('Dataset')
  
  image.save("/content/imagetest.jpeg")
  video.save("/content/videotest.mp4")
  source_image_path = '/content/imagetest.jpeg'
  driving_video_path = '/content/videotest.mp4'
  output_video_path = '/content/imagetest_videotest.mp4'

  source_image = imageio.imread(source_image_path)
  reader = imageio.get_reader(driving_video_path)
  source_image = resize(source_image, (pixel, pixel))[..., :3]

  if(dataset == 'fashion'):
    fps = reader.get_meta_data()['fps']

  driving_video = []
  try:
      for im in reader:
          driving_video.append(im)
  except RuntimeError:
      pass
  reader.close()
  driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

  from first.demo import make_animation
  from skimage import img_as_ubyte
  if(dataset == 'fashion'):
    predictions = make_animation(source_image, driving_video, generatorFashion, kp_detectorFashion, relative=True)
  else:
    predictions = make_animation(source_image, driving_video, generatorGif, kp_detectorGif, relative=True)
   #save resulting video
  if(dataset == 'fashion'):
    imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps = fps)
  else:
    imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions])

  if os.path.exists(output_video_path):
      print("Success")
      return send_file(output_video_path)
  else:
      return 'Error'


server.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c120-34-67-61-106.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Ana fel function


  0%|          | 0/425 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/May/2023 14:19:59] "POST /post HTTP/1.1" 200 -


Success
Ana fel function


  0%|          | 0/17 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/May/2023 14:21:21] "POST /post HTTP/1.1" 200 -


Success
